In [1]:
!pip install datasets
!pip install transformers
!pip install py7zr
!pip install sentencepiece
!pip install rouge-score
!pip install bert-score
!pip install git+https://github.com/google-research/bleurt.git
!pip install nltk
!pip install rouge
!pip install bleurt
!pip install sentencepiece
!pip install evaluate
!wget https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
!unzip bleurt-base-128.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from bleurt import score
from bert_score import BERTScorer
import numpy as np
from nltk.translate.bleu_score import sentence_bleu


from transformers import T5Tokenizer


In [3]:
# Load the dataset
dataset = load_dataset("samsum")
train_data = dataset["train"]
val_data = dataset["validation"]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [4]:
# Load the pre-trained T5-base model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available 

In [5]:
def preprocess_function(examples):
    inputs = [f'dialogue: {d}' for d in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
# Apply the preprocessing function to the train and validation data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)



# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [7]:
def collate_fn(batch):
    input_ids = []
    attention_mask = []
    labels = []

    for item in batch:
        input_ids.append(torch.tensor(item["input_ids"]))
        attention_mask.append(torch.tensor(item["attention_mask"]))
        labels.append(torch.tensor(item["labels"]))

    return {
        "input_ids": torch.stack(input_ids, dim=0),
        "attention_mask": torch.stack(attention_mask, dim=0),
        "labels": torch.stack(labels, dim=0)
    }

# Create data loaders
batch_size = 8  # Define the batch size as desired
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode for each epoch
    total_loss = 0.0
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss}")

    model.eval()
    total_val_loss = 0.0
    for batch in val_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}")


Epoch 1/5, Training Loss: 0.465937429430301
Epoch 1/5, Validation Loss: 0.33255046417990936
Epoch 2/5, Training Loss: 0.34170072939121504
Epoch 2/5, Validation Loss: 0.3238290804393083
Epoch 3/5, Training Loss: 0.31122785128337555
Epoch 3/5, Validation Loss: 0.32215908632695095
Epoch 4/5, Training Loss: 0.28435683273482787
Epoch 4/5, Validation Loss: 0.3241387362619048
Epoch 5/5, Training Loss: 0.2597707704834066
Epoch 5/5, Validation Loss: 0.3302574821756881


In [8]:
test_data = dataset["test"]
test_data = test_data.map(preprocess_function, batched=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [9]:
# Save the fine-tuned model after all epochs
model.save_pretrained('t5-base')

In [10]:
# Function to generate summaries using the fine-tuned model
def summarize_text(text, model, tokenizer, device):
    # Input Tokenization
    input_ids = tokenizer.encode(text, truncation=True, max_length=256, return_tensors='pt')

    # Move input_ids to device
    input_ids = input_ids.to(device)

    # Generating summary
    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Decoding the summary tokens back to text
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    return summary


In [11]:
# Select a sample from the dataset
sample = dataset['test'][0]
input_text = sample['dialogue']

In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary = summarize_text(input_text, model, tokenizer,device)


In [14]:
  print("Summary:", summary)

Summary: Amanda can't find Betty's number. Hannah doesn't know Larry. Amanda will text Larry.


In [16]:
# Iniatiate References,Predictions and feed them

references = []  # List to store the ground truth summaries
predictions = []  # List to store the predicted summaries

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    # Convert the tensors to text using the tokenizer
    inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    targets = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Add the texts to the corresponding lists
    references.extend(targets)

    # Generate the summaries and add them to the list
    with torch.no_grad():
        output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(outputs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
print(f"Number of predictions: {len(predictions)}")
print(f"Number of references: {len(references)}")


Number of predictions: 819
Number of references: 819


In [18]:
import evaluate
bleu = evaluate.load('bleu')
#bleu_score = bleu.compute(references=refs, predictions=preds)

In [22]:
from rouge import Rouge


In [23]:


# Instantiate the metric calculators
rouge = Rouge()
bleurt_scorer = score.BleurtScorer("bleurt-base-128")
#bleurt_scorer = score.BleurtScorer('bleurt/test_checkpoint')
bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
total_scores = {"bleu":0}

# Calculate ROUGE
rouge_scores = rouge.get_scores(predictions, references, avg=True)
rouge1_f = rouge_scores['rouge-1']['f']
rouge2_f = rouge_scores['rouge-2']['f']
rougeL_f = rouge_scores['rouge-l']['f']
overall_rouge = (rouge1_f + rouge2_f + rougeL_f) / 3

# Calculate BLEURT
bleurt_scores = bleurt_scorer.score(references=references, candidates=predictions)
mean_bleurt_score = np.mean(bleurt_scores)

# Calculate BERTScore
P, R, F1 = bert_scorer.score(predictions, references)
mean_bert_score = F1.mean().item()

 # BLEU Scores
    #bleu_scores = [sentence_bleu([ref], pred, weights=(1.0, 0, 0, 0)) for ref, pred in zip(real_summaries, pred_summaries)]
total_scores["bleu"] += bleu.compute(references=references, predictions=predictions)['bleu']






Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
print("ROUGE-1 F-Measure:", rouge1_f)
print("ROUGE-2 F-Measure:", rouge2_f)
print("ROUGE-L F-Measure:", rougeL_f)
print("Overall ROUGE Score:", overall_rouge)
print("BLEURT Score:", mean_bleurt_score)
print('BERTScore: {:.4f}'.format(mean_bert_score))
print('BLEU Score: {:.4f}'.format(total_scores['bleu']))


ROUGE-1 F-Measure: 0.44557217657352677
ROUGE-2 F-Measure: 0.20504069814805864
ROUGE-L F-Measure: 0.41399679800347117
Overall ROUGE Score: 0.35486989090835214
BLEURT Score: -0.4449399154364917
BERTScore: 0.4598
BLEU Score: 0.1160
